In [2]:
%%capture
!pip install nltk datasets

# BERTopic Analysis - Court Rulings (Conflict of Interests)

In [ ]:
import nltk
from nltk.tokenize.punkt import PunktTrainer, PunktSentenceTokenizer
from datasets import load_dataset

## Data
Import the dataset from Hugging Face 🤗.

In [4]:
data = load_dataset("istat-ai/court-rulings-coi", split="train")

## Sentence Tokenization
Now, we need to split the full rulings into individual phrases. To do this, we train a custom tokenizer.

In [ ]:
long_text = ""

for text in data["Clean_Text"]:
    long_text += text

trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(long_text)

tokenizer = PunktSentenceTokenizer(trainer.get_params())

Now we can apply it to the texts.

In [20]:
doc_ids = []
all_sentences = []

for idx, text in enumerate(data["Clean_Text"]):
    sentences = tokenizer.tokenize(text)
    all_sentences.extend(sentences)
    doc_ids.extend([idx] * len(sentences))

---

## BERTopic Modeling